<a href="https://colab.research.google.com/github/extrajp2014/DS-Unit-1-Sprint-2-Data-Wrangling/blob/master/module2-join-datasets/LS_DS_122_Join_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

_Lambda School Data Science_

# Join datasets

Objectives
- concatenate data with pandas
- merge data with pandas

Links
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)
  - Combine Data Sets: Standard Joins
- Python Data Science Handbook
  - [Chapter 3.6](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html), Combining Datasets: Concat and Append
  - [Chapter 3.7](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html), Combining Datasets: Merge and Join

## Download data

We’ll work with a dataset of [3 Million Instacart Orders, Open Sourced](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2)!

In [2]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2019-01-15 19:59:22--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.236.85
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.236.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.2’

instacart_online_gr 100%[===================>] 196.03M  78.1MB/s    in 2.5s    

2019-01-15 19:59:25 (78.1 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.2’ saved [205548478/205548478]



In [3]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [4]:
%cd instacart_2017_05_01
!ls | wc

/content/instacart_2017_05_01
      8       8     176


## Goal: Reproduce this example

The first two orders for user id 1:

In [49]:
from IPython.display import display, Image
url = 'https://cdn-images-1.medium.com/max/1600/1*vYGFQCafJtGBBX5mbl0xyw.png'
example = Image(url=url, width=600)

display(example)

## Load data

Here's a list of all six CSV filenames

In [6]:
!ls -lh

total 681M
-rw-r--r-- 1 502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 502 staff  270 May  2  2017 departments.csv
-rw-r--r-- 1 502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1 502 staff 2.1M May  2  2017 products.csv


For each CSV
- Load it with pandas
- Look at the dataframe's shape
- Look at its head (first rows)
- `display(example)`
- Which columns does it have in common with the example we want to reproduce?

### aisles

In [0]:
import pandas as pd
aisles = pd.read_csv('aisles.csv')

In [8]:
aisles.shape

(134, 2)

In [9]:
aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [10]:
display(example)

aisles doesn't have any columns we need

### departments

In [0]:
departments = pd.read_csv('departments.csv')

In [12]:
departments.shape

(21, 2)

In [13]:
departments.sample(n=5)

,department_id,department
14,15,canned goods
9,10,bulk
16,17,household
2,3,bakery
13,14,breakfast


In [14]:
display(example)

departments doesn't have any columns we need

### order_products__prior

In [0]:
order_products__prior = pd.read_csv('order_products__prior.csv')

In [16]:
order_products__prior.shape

(32434489, 4)

In [17]:
order_products__prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


This dataframe has order_id, product_id, add_to_cart_order

### order_products__train

In [0]:
order_products__train = pd.read_csv('order_products__train.csv')

In [19]:
order_products__train.shape

(1384617, 4)

In [20]:
order_products__train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


This dataframe has the exact same columns as the prior one

### orders

In [0]:
orders = pd.read_csv('orders.csv')

In [22]:
orders.shape

(3421083, 7)

In [23]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


This has user_id, order_id, order_number, order_dow, order_hour_of_day

### products

In [0]:
products = pd.read_csv('products.csv')

In [25]:
products.shape

(49688, 4)

In [26]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


products has product_id, product_name

## Concatenate order_products__prior and order_products__train

In [70]:
order_products = pd.concat([order_products__prior, order_products__train])
order_products.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [28]:
order_products.shape, order_products__prior.shape, order_products__train.shape

((33819106, 4), (32434489, 4), (1384617, 4))

In [0]:
assert len(order_products) == len(order_products__prior) + len(order_products__train)

In [73]:
assert (len(order_products.columns) == 
        len(order_products__prior.columns) == 
        len(order_products__train.columns))

print(order_products.shape, order_products__prior.shape, order_products__train.shape)
order_products.head()

(33819106, 4) (32434489, 4) (1384617, 4)


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


## Get a subset of orders — the first two orders for user id 1

From the `orders` dataframe, with these columns:
- user_id
- order_id
- order_number
- order_dow
- order_hour_of_day

In [66]:
condition = (orders.user_id==1) & (orders.order_number <= 2)

columns = ['user_id', 
           'order_id', 
           'order_number', 
           'order_dow', 
           'order_hour_of_day']

subset = orders[columns]
subset.shape


(3421083, 5)

## Merge dataframes

In [0]:
columns = ['order_id', 'add_to_cart_order', 'product_id']

merged = pd.merge(subset, order_products[columns], 
                  how='inner', on='order_id')

In [33]:
subset.shape, merged.shape

((2, 5), (11, 7))

In [34]:
subset

,user_id,order_id,order_number,order_dow,order_hour_of_day
0,1,2539329,1,2,8
1,1,2398795,2,3,7


In [35]:
merged

,user_id,order_id,order_number,order_dow,order_hour_of_day,add_to_cart_order,product_id
0,1,2539329,1,2,8,1,196
1,1,2539329,1,2,8,2,14084
2,1,2539329,1,2,8,3,12427
3,1,2539329,1,2,8,4,26088
4,1,2539329,1,2,8,5,26405
5,1,2398795,2,3,7,1,196
6,1,2398795,2,3,7,2,10258
7,1,2398795,2,3,7,3,12427
8,1,2398795,2,3,7,4,13176
9,1,2398795,2,3,7,5,26088


In [36]:
display(example)

1. Add `product_name`

In [0]:
final = pd.merge(merged, products[['product_id', 'product_name']])

In [0]:
final = final.sort_values(by=['order_number', 'add_to_cart_order'])

In [39]:
display(example)

2. Rename columns so they have spaces instead of underscores

In [0]:
final.columns = [column.replace('_', ' ') 
                 for column in final]

In [41]:
final.head()

,user id,order id,order number,order dow,order hour of day,add to cart order,product id,product name
0,1,2539329,1,2,8,1,196,Soda
2,1,2539329,1,2,8,2,14084,Organic Unsweetened Vanilla Almond Milk
3,1,2539329,1,2,8,3,12427,Original Beef Jerky
5,1,2539329,1,2,8,4,26088,Aged White Cheddar Popcorn
7,1,2539329,1,2,8,5,26405,XL Pick-A-Size Paper Towel Rolls


# Assignment

These are the top 10 most frequently ordered products. How many times was each ordered? 

1. Banana
2. Bag of Organic Bananas
3. Organic Strawberries
4. Organic Baby Spinach 
5. Organic Hass Avocado
6. Organic Avocado
7. Large Lemon 
8. Strawberries
9. Limes 
10. Organic Whole Milk

First, write down which columns you need and which dataframes have them.

Next, merge these into a single dataframe.

Then, use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.

## Stretch challenge

The [Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) has a visualization of "**Popular products** purchased earliest in the day (green) and latest in the day (red)." 

The post says,

> "We can also see the time of day that users purchase specific products.

> Healthier snacks and staples tend to be purchased earlier in the day, whereas ice cream (especially Half Baked and The Tonight Dough) are far more popular when customers are ordering in the evening.

> **In fact, of the top 25 latest ordered products, the first 24 are ice cream! The last one, of course, is a frozen pizza.**"

Your challenge is to reproduce the list of the top 25 latest ordered popular products.

We'll define "popular products" as products with more than 2,900 orders.

In [2]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz
%cd instacart_2017_05_01
!ls | wc
!pwd

--2019-01-15 20:13:24--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.133.109
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.133.109|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’

instacart_online_gr 100%[===================>] 196.03M  43.6MB/s    in 4.9s    

2019-01-15 20:13:29 (39.6 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’ saved [205548478/205548478]

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_

In [0]:
# LOAD DATA
import pandas as pd
order_products__prior = pd.read_csv('order_products__prior.csv')
order_products__train = pd.read_csv('order_products__train.csv')
orders = pd.read_csv('orders.csv')
products = pd.read_csv('products.csv')

In [4]:
#COMBINE DATA = order_products
# Concatenate order_products__prior and order_products__train
order_products = pd.concat([order_products__prior, order_products__train])
print(order_products.shape, order_products__prior.shape, order_products__train.shape)
order_products.head()

(33819106, 4) (32434489, 4) (1384617, 4)


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [5]:
# Testing
assert len(order_products) == len(order_products__prior) + len(order_products__train)

assert (len(order_products.columns) == 
        len(order_products__prior.columns) == 
        len(order_products__train.columns))
print(order_products.shape, order_products__prior.shape, order_products__train.shape)
order_products.head()

(33819106, 4) (32434489, 4) (1384617, 4)


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [7]:
#SUBSET of orders df = subset
columns = ['user_id', 
           'order_id', 
           'order_number', 
           'order_dow', 
           'order_hour_of_day']
subset = orders[columns]
print(subset.shape)
subset.head()

(3421083, 5)


,user_id,order_id,order_number,order_dow,order_hour_of_day
0,1,2539329,1,2,8
1,1,2398795,2,3,7
2,1,473747,3,3,12
3,1,2254736,4,4,7
4,1,431534,5,4,15


In [9]:
#MERGE subset to order_products df = merged
columns = ['order_id', 'add_to_cart_order', 'product_id']
merged = pd.merge(subset, order_products[columns], how='inner', on='order_id')
print(merged.shape)
merged.head()

(33819106, 7)


,user_id,order_id,order_number,order_dow,order_hour_of_day,add_to_cart_order,product_id
0,1,2539329,1,2,8,1,196
1,1,2539329,1,2,8,2,14084
2,1,2539329,1,2,8,3,12427
3,1,2539329,1,2,8,4,26088
4,1,2539329,1,2,8,5,26405


In [10]:
#MERGE order_products to products[id and name]  = merged
final = pd.merge(merged, products[['product_id', 'product_name']])
final = final.sort_values(by=['order_number', 'add_to_cart_order'])
print(final.shape)
final.head()

(33819106, 8)


,user_id,order_id,order_number,order_dow,order_hour_of_day,add_to_cart_order,product_id,product_name
0,1,2539329,1,2,8,1,196,Soda
38,67,3205960,1,3,11,1,196,Soda
97,360,3226575,1,5,12,1,196,Soda
136,793,231498,1,1,10,1,196,Soda
146,991,2696986,1,3,16,1,196,Soda


In [26]:
# Banana
# Bag of Organic Bananas
# Organic Strawberries
# Organic Baby Spinach
# Organic Hass Avocado
# Organic Avocado
# Large Lemon
# Strawberries
# Limes
# Organic Whole Milk
final['product_name'].value_counts()[0:10]

Banana                    491291
Bag of Organic Bananas    394930
Organic Strawberries      275577
Organic Baby Spinach      251705
Organic Hass Avocado      220877
Organic Avocado           184224
Large Lemon               160792
Strawberries              149445
Limes                     146660
Organic Whole Milk        142813
Name: product_name, dtype: int64

In [0]:
# STRETCH
# Your challenge is to reproduce the list of the top 25 latest ordered popular products.
# We'll define "popular products" as products with more than 2,900 orders.
# print(final[final['product_name'] > 2900].shape)
valueCount=final['product_name'].value_counts()
